In [71]:
import wikipedia
import pandas as pd
from tqdm import tqdm 
import math
import numpy as np
import html
import re
from unidecode import unidecode
import pickle

In [3]:
df = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/anime-dataset-2023.csv")
df2 = pd.read_excel("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/combined_final_anime.xlsx", index_col=0)

In [118]:
df[df['Name'] == 'Tonari no Totoro']

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
490,523,Tonari no Totoro,My Neighbor Totoro,となりのトトロ,8.25,"Adventure, Award Winning, Supernatural","In 1950s Japan, Tatsuo Kusakabe relocates hims...",Movie,1.0,"Apr 16, 1988",UNKNOWN,Finished Airing,"Studio Fantasia, Streamline Pictures","Disney Platform Distribution, GKIDS",Studio Ghibli,Original,1 hr 26 min,G - All Ages,287.0,143,7978,658094.0,1009153,https://cdn.myanimelist.net/images/anime/4/759...


In [20]:
pd.set_option('display.max_columns', None)

In [59]:
df_add_on = df[df['Type'].isin(['TV', 'Movie', 'OVA', 'Special', 'ONA'])]

In [45]:
merge_df = df2.merge(df[['anime_id', 'Producers', 'Licensors', 'Studios', 'Image URL', 'Episodes', 'Genres', 'Source', 'Favorites', 'Aired', 'Members', 'Duration']], left_on = 'anime_anime_id', right_on = 'anime_id', how = 'left')

In [46]:
merge_df['Producers'] = merge_df['Producers'].replace("UNKNOWN", np.nan)
merge_df['Licensors'] = merge_df['Licensors'].replace("UNKNOWN", np.nan)
merge_df['Studios'] = merge_df['Studios'].replace("UNKNOWN", np.nan)
merge_df['Duration'] = merge_df['Duration'].replace("UNKNOWN", np.nan)

In [47]:
merge_df['Producers'] = merge_df['Producers'].fillna(merge_df['anime_Producers'])
merge_df['Licensors'] = merge_df['Licensors'].fillna(merge_df['anime_Licensors'])
merge_df['Studios'] = merge_df['Studios'].fillna(merge_df['anime_Studios'])
merge_df['Duration'] = merge_df['Duration'].fillna(merge_df['anime_Duration'])

In [49]:
merge_df.to_excel('combo_fin_anime_v2.xlsx')

In [52]:
merge_df.replace({'none': 'UNKNOWN', 'None': 'UNKNOWN', np.nan: 'UNKNOWN'})


,imdb_title_title,imdb_title_titleId,anime_Aired,anime_Rating,anime_Producers,anime_Studios,anime_Licensors,anime_Duration,anime_English name,anime_Genres,anime_Image URL,anime_Other name,anime_Score,anime_Synopsis,anime_anime_id,imdb_name_basics_primaryName,plot,keywords,image,text,anime_id,Producers,Licensors,Studios,Image URL,Episodes,Genres,Source,Favorites,Aired,Members,Duration
0,【推しの子】,tt21030032,"Apr 12, 2023 to Jun 28, 2023",PG-13 - Teens 13 or older,"Shueisha, CyberAgent, Kadokawa",Doga Kobo,Sentai Filmworks,30 min,[Oshi No Ko],"Drama, Supernatural",UNKNOWN,【推しの子】,8.98,"In the entertainment world, celebrities often ...",52034,"Natalie Rial, Christina Marie Kelly, Takeo Ots...","Oshi no Ko (【推しの子】, ""My Favorite Idol"" or ""The...","idol,anime animation,based on manga,singer,per...",https://m.media-amazon.com/images/M/MV5BYWFkZD...,"Oshi no Ko (【推しの子】, ""My Favorite Idol"" or ""The...",52034.0,"Shueisha, CyberAgent, Kadokawa",Sentai Filmworks,Doga Kobo,https://cdn.myanimelist.net/images/anime/1812/...,11.0,"Drama, Supernatural",Manga,18336.0,"Apr 12, 2023 to Jun 28, 2023",512617.0,30 min per ep
1,.hack//G.U. Returner,tt1343075,2007-01-18 00:00:00,PG-13 - Teens 13 or older,"Bandai Visual, CyberConnect2",Bee Train,UNKNOWN,24 min,UNKNOWN,"Adventure, Drama, Fantasy",UNKNOWN,.HACK//G.U. RETURNER,6.67,The characters from previous .hack//G.U. Games...,2928,"Kaori Nazuka, Ayako Kawasumi, Takahiro Sakurai...",.hack//G.U. is a series of single-player actio...,.hack,https://m.media-amazon.com/images/M/MV5BYjViMD...,.hack//G.U. is a series of single-player actio...,2928.0,"Bandai Visual, CyberConnect2",UNKNOWN,Bee Train,https://cdn.myanimelist.net/images/anime/1798/...,1.0,"Adventure, Drama, Fantasy",Game,32.0,"Jan 18, 2007",21271.0,24 min
2,.hack//G.U. Trilogy,tt1164545,2007-12-22 00:00:00,PG-13 - Teens 13 or older,Bandai Visual,CyberConnect2,"Funimation, Bandai Entertainment",1 hr 33 min,.hack//G.U. Trilogy: Parody Mode,"Comedy, Fantasy, Sci-Fi",UNKNOWN,.hack//G.U. Trilogy,6.36,A special bonus Parody Mode added to the extra...,4469,"Hiroshi Matsuyama, Ayako Kawasumi, Saeko Hirot...",.hack//G.U. Trilogy (Japanese: ドットハック G.U. Tri...,.hack,https://m.media-amazon.com/images/M/MV5BODU4Nz...,.hack//G.U. Trilogy (Japanese: ドットハック G.U. Tri...,4469.0,Bandai Visual,"Funimation, Bandai Entertainment",CyberConnect2,https://cdn.myanimelist.net/images/anime/10/86...,1.0,"Comedy, Fantasy, Sci-Fi",Game,10.0,"Mar 25, 2008",10641.0,6 min
3,.hack//GIFT,tt0823406,"Nov 16, 2003",R+ - Mild Nudity,CyberConnect2,Bee Train,Bandai Entertainment,26 min,.hack//Gift,"Comedy, Fantasy",https://cdn.myanimelist.net/images/anime/2/230...,.hack//GIFT,6.1,As an expression of gratitude for the heroes o...,454,"Isshin Chiba, Shin'ya Kawatsura, Sayaka Aida, ...",This is a list of media related to the .hack m...,"anime animation,parody comedy,spoof,hot spring...",https://m.media-amazon.com/images/M/MV5BNjUwMT...,As an expression of gratitude for the heroes o...,454.0,CyberConnect2,Bandai Entertainment,Bee Train,https://cdn.myanimelist.net/images/anime/2/230...,1.0,"Comedy, Fantasy",Original,19.0,"Nov 16, 2003",20567.0,26 min
4,.hack//SIGN,tt0361140,"Apr 4, 2002 to Sep 26, 2002",PG-13 - Teens 13 or older,"Bandai Visual, Yomiko Advertising, Bandai, ...",Bee Train,"Funimation, Bandai Entertainment",24 min,.hack//Sign,"Adventure, Fantasy, Mystery",UNKNOWN,.hack//SIGN,6.95,"A young wavemaster, only known by the alias of...",48,"Amanda Winn Lee, Kaori Nazuka, Donna Rawlins, ...",.hack//Sign (stylized as .hack//SIGN) is a Jap...,"isekai,anime animation,female nudity,multiplay...",https://m.media-amazon.com/images/M/MV5BOTE5YW...,.hack//Sign (stylized as .hack//SIGN) is a Jap...,48.0,"Bandai Visual, Yomiko Advertising, Bandai, Cyb...","Funimation, Bandai Entertainment",Bee Train,https://cdn.myanimelist.net/images/anime/1443/...,26.0,"Adventure, Fantasy, Mystery",Original,1343.0,"Apr 4, 2002 to Sep 26, 2002",178659.0,24 min per ep
...,...,...,...,...,...,...,...,...,...,...,

In [67]:
df_add_on = df_add_on[~df_add_on['anime_id'].isin(merge_df['anime_anime_id'].tolist())].reset_index(drop = True)

In [74]:
# Function to get Wikipedia content for a given title
def get_wikipedia_content(title):
    import time

    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        # If the title is ambiguous, you can handle it as per your requirement
        return f"Ambiguous title: {title}. Options: {e.options}"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for: {title}"
    except wikipedia.exceptions.HTTPTimeoutError as e:
        return f"Timeout error occurred: {str(e)}"
    except wikipedia.exceptions.WikipediaException as e:
        if "HTTP 443" in str(e):
            print("Received HTTP 443 error. Pausing for 2 minutes...")
            time.sleep(120)  # Pause for 2 minutes (120 seconds)
            print("Resuming...")
            return get_wikipedia_content(title)  # Retry
        else:
            return f"Error occurred: {str(e)}"
    except Exception as e:
        return f"Error occurred: {str(e)}"
    
def fetch_wikipedia_content(df, get_wikipedia_content_func):
    # Dictionary to store data
    data_dict = {}

    # Loop through each title in the DataFrame with tqdm for progress tracking
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Fetching Wikipedia Content"):
        imdb_id = row['anime_id']
        title = row['Name']

        # Get Wikipedia content for the title
        content = get_wikipedia_content_func(title)

        # Add data to dictionary
        if imdb_id not in data_dict:
            data_dict[imdb_id] = {'Title': title, 'Content': content}
        else:
            print(f"Duplicate IMDb ID found: {imdb_id}")

    return data_dict

In [75]:
df_add_on.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [76]:
len(df_add_on)

17058

In [77]:
pull1 = df_add_on.iloc[0:1500,:]
pull2 = df_add_on.iloc[1500:3000,:]
pull3 = df_add_on.iloc[3000:4500,:]
pull4 = df_add_on.iloc[4500:6000,:]
pull5 = df_add_on.iloc[6000:7500,:]
pull6 = df_add_on.iloc[7500:9000,:]
pull7 = df_add_on.iloc[9000:10500,:]
pull8 = df_add_on.iloc[10500:12000,:]
pull9 = df_add_on.iloc[12000:13500,:]
pull10 = df_add_on.iloc[13500:15000,:]
pull11 = df_add_on.iloc[15000:16500,:]
pull12 = df_add_on.iloc[16500:,:]

In [78]:
fetch1 = fetch_wikipedia_content(pull1, get_wikipedia_content)

Fetching Wikipedia Content:   3%|▎         | 41/1500 [00:39<23:49,  1.02it/s]/Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Fetching Wikipedia Content: 100%|██████████| 1500/1500 [23:54<00:00,  1.05it/s]


In [79]:
fetch2 = fetch_wikipedia_content(pull2, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [23:19<00:00,  1.07it/s] 


In [80]:
fetch3 = fetch_wikipedia_content(pull3, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [22:43<00:00,  1.10it/s]


In [81]:
fetch4 = fetch_wikipedia_content(pull4, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [21:50<00:00,  1.14it/s] 


In [82]:
fetch5 = fetch_wikipedia_content(pull5, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [20:17<00:00,  1.23it/s]


In [83]:
fetch6 = fetch_wikipedia_content(pull6, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [20:28<00:00,  1.22it/s]


In [84]:
fetch7 = fetch_wikipedia_content(pull7, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [19:57<00:00,  1.25it/s] 


In [85]:
fetch8 = fetch_wikipedia_content(pull8, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [20:05<00:00,  1.24it/s]


In [86]:
fetch9 = fetch_wikipedia_content(pull9, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [19:24<00:00,  1.29it/s] 


In [87]:
fetch10 = fetch_wikipedia_content(pull10, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [18:47<00:00,  1.33it/s]


In [88]:
fetch11 = fetch_wikipedia_content(pull11, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1500/1500 [20:26<00:00,  1.22it/s] 


In [89]:
fetch12 = fetch_wikipedia_content(pull12, get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 558/558 [07:25<00:00,  1.25it/s]


In [90]:
# Combine dictionaries
combined_dict = {}
combined_dict.update(fetch1)
combined_dict.update(fetch2)
combined_dict.update(fetch3)
combined_dict.update(fetch4)
combined_dict.update(fetch5)
combined_dict.update(fetch6)
combined_dict.update(fetch7)
combined_dict.update(fetch8)
combined_dict.update(fetch9)
combined_dict.update(fetch10)
combined_dict.update(fetch11)
combined_dict.update(fetch12)

In [91]:
import pickle

# File path to save the dictionary
file_path = "anime_wiki_dict.pkl"

# Save the dictionary using pickle
with open(file_path, 'wb') as f:
    pickle.dump(combined_dict, f)

In [92]:
df_wiki = pd.DataFrame(combined_dict).T

In [97]:
filtered_df_wiki2 = df_wiki[~df_wiki['Content'].str.startswith('No Wikipedia page found for:')].reset_index()

In [123]:
filtered_df_wiki2

,index,Title,Content
0,18,Initial D Fourth Stage,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis..."
1,23,Ring ni Kakero 1,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri..."
2,62,D.C.: Da Capo,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate..."
3,63,DearS,Bears are carnivoran mammals of the family Urs...
4,102,Aishiteruze Baby★★,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei..."
...,...,...,...
6719,55719,The Demon Queen Has a Death Wish,Mage & Demon Queen is a fantasy-comedy webtoon...
6720,55721,See You on Our Veranda,Emeril John Lagassé III ( EM-ə-rəl lə-GAH-see;...
6721,55722,No More Princes,The Princes in the Tower refers to the mystery...
6722,55723,The Forgotten Princess Just Wants Peace,This article is a list of episodes from the te...


In [126]:
add_on_fin_df = df_add_on.merge(filtered_df_wiki2, left_on = 'anime_id', right_on = 'index', how = 'inner')

In [128]:
# Define the regular expression pattern
pattern = r'^.*?\n\n\n(.*?)\n\n\n'
add_on_fin_df['match'] = add_on_fin_df['Content'].apply(lambda x: re.search(pattern, x, re.DOTALL))
add_on_fin_df['plot'] = add_on_fin_df['match'].apply(lambda x: x.group(0)if x else "None")

In [130]:
add_on_fin_df = add_on_fin_df[add_on_fin_df['plot'] != "None"]

In [134]:
print(add_on_fin_df.tail(1)['plot'])

6722    This article is a list of episodes from the te...
Name: plot, dtype: object


In [138]:
add_on_fin_df = add_on_fin_df[~add_on_fin_df['plot'].str.startswith("This article is a")]


In [143]:
add_on_fin_df['plot'] = add_on_fin_df['plot'].str.replace('== Plot ==', '')

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_24741/2032242105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_on_fin_df['plot'] = add_on_fin_df['plot'].str.replace('== Plot ==', '')


In [145]:
merge_df.columns

Index(['imdb_title_title', 'imdb_title_titleId', 'anime_Aired', 'anime_Rating',
       'anime_Producers', 'anime_Studios', 'anime_Licensors', 'anime_Duration',
       'anime_English name', 'anime_Genres', 'anime_Image URL',
       'anime_Other name', 'anime_Score', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords', 'image', 'text',
       'anime_id', 'Producers', 'Licensors', 'Studios', 'Image URL',
       'Episodes', 'Genres', 'Source', 'Favorites', 'Aired', 'Members',
       'Duration'],
      dtype='object')

In [154]:
# Define a function to compare the lengths of 'text' and 'anime_Synopsis' columns and return the appropriate value
def replace_text(row):
    if len(row['plot']) < len(row['Synopsis']):
        return row['Synopsis']
    else:
        return row['plot']

In [155]:
add_on_fin_df['text'] = add_on_fin_df.apply(replace_text, axis=1)

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_24741/470037777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_on_fin_df['text'] = add_on_fin_df.apply(replace_text, axis=1)


In [157]:
add_on_fin_df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL',
       'index', 'Title', 'Content', 'match', 'plot', 'text'],
      dtype='object')

In [160]:
add_on_fin_df['imdb_title_title'] = 'UNKNOWN'
add_on_fin_df['imdb_title_titleId'] = 'UNKNOWN'
add_on_fin_df['anime_Rating'] = add_on_fin_df['Rating']
add_on_fin_df['anime_Genres'] = add_on_fin_df['Genres']
add_on_fin_df['anime_Other name'] = add_on_fin_df['Other name']
add_on_fin_df['anime_English name'] = add_on_fin_df['English name']
add_on_fin_df['anime_Synopsis'] = add_on_fin_df['Synopsis']
add_on_fin_df['anime_Score'] = add_on_fin_df['Score']
add_on_fin_df['imdb_name_basics_primaryName'] = 'UNKNOWN'
add_on_fin_df['anime_anime_id'] = add_on_fin_df['anime_id']
add_on_fin_df['keywords'] = 'UNKNOWN'
add_on_fin_df['image'] = 'UNKNOWN'

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_24741/2554166314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_on_fin_df['imdb_title_title'] = 'UNKNOWN'
/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_24741/2554166314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_on_fin_df['imdb_title_titleId'] = 'UNKNOWN'
/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_24741/2554166314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [194]:
fin_fin_df = pd.concat([add_on_fin_df[['imdb_title_title', 'imdb_title_titleId', 'anime_Rating',
       'anime_English name', 'anime_Genres',
       'anime_Other name', 'anime_Score', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords', 'image', 'Producers', 'Licensors', 'Studios', 'Image URL',
       'Episodes', 'Genres', 'Source', 'Favorites', 'Aired', 'Members',
       'Duration', 'text']],
merge_df[['imdb_title_title', 'imdb_title_titleId', 'anime_Rating',
       'anime_English name', 'anime_Genres',
       'anime_Other name', 'anime_Score', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords', 'image', 'Producers', 'Licensors', 'Studios', 'Image URL',
       'Episodes', 'Genres', 'Source', 'Favorites', 'Aired', 'Members',
       'Duration', 'text']]], ignore_index = True)

In [195]:
fin_fin_df = fin_fin_df.replace({'none': 'UNKNOWN', 'None': 'UNKNOWN', np.nan: 'UNKNOWN'})

In [196]:
fin_fin_df = fin_fin_df.merge(df[['Name', 'anime_id']], left_on='anime_anime_id', right_on = 'anime_id')

In [197]:
fin_fin_df.to_json('fin_anime_df.json')

In [198]:
myanime_list = fin_fin_df['anime_id'].tolist()

In [199]:
file_path = "my_animelist.pkl"

# Open the file in binary write mode and save the list using pickle.dump()
with open(file_path, 'wb') as f:
    pickle.dump(myanime_list, f)